# Modelo planilha quantitativa

Cria duas planilhas, uma com dados quantitativos de beneficiarios com "MÓDULO" igual a "APT" ("AMB HOSP OBST APT") e outra planilha com os que possuem "MÓDULO" igual a "ENF" ("AMB HOSP OBST ENF")

## As colunas do DataFrame orignal:
CARTEIRA, MATRÍCULA, DEPENDENCIA, NOME, CPF, NASCIMENTO, SEXO, ESTADO CIVIL, CÓD MÓDULO, MÓDULO, INICIO VIGÊNCIA, LOTAÇÃO e REPASSE

## Colunas importantes:
DEPENDENCIA, NOME, CPF, NASCIMENTO, SEXO e MÓDULO

### Informção importante:
NASCIMENTO - dd/mm/aaaa

# Modelo planilhas a serem criadas (ENF e APT):

| Idade      | Titulares e dependentes | Agregados |
| :---       | :---                    | :---      |
| 00-18      |                         |           |
| 19-23      |                         |           |
| 24-28      |                         |           |
| 29-33      |                         |           |
| 34-38      |                         |           |
| 39-43      |                         |           |
| 44-48      |                         |           |
| 49-53      |                         |           |
| 54-58      |                         |           |
| 59 ou mais |                         |           |

In [ ]:
import pandas as pd

dados = pd.read_csv("")
dados.head()

In [ ]:
# Filtrar linhas onde NOME não é nulo
df_filtrado = dados[dados['NOME'].notna()].copy()

# Converter coluna NASCIMENTO para datetime
df_filtrado['NASCIMENTO'] = pd.to_datetime(df_filtrado['NASCIMENTO'], dayfirst=True, errors='coerce')

# Calcular idade com base na data atual
hoje = pd.Timestamp.now()
df_filtrado['Idade'] = ((hoje - df_filtrado['NASCIMENTO']).dt.days / 365.25).fillna(-1).astype(int)

# Definir as faixas etárias
bins = [-1, 18, 23, 28, 33, 38, 43, 48, 53, 58, 200]
labels = ['00-18', '19-23', '24-28', '29-33', '34-38', '39-43', '44-48', '49-53', '54-58', '59 ou mais']
df_filtrado['Faixa Etaria'] = pd.cut(df_filtrado['Idade'], bins=bins, labels=labels)

# Classificar beneficiários (Titulares/Dependentes vs Agregados)
# Assumindo que 'Agregado' na coluna DEPENDENCIA define os Agregados, e o resto são Titulares/Dependentes
df_filtrado['Classificacao'] = df_filtrado['DEPENDENCIA'].apply(
    lambda x: 'Agregados' if x == 'Agregado' else 'Titulares e dependentes'
)

# Função para criar o DataFrame formatado
def criar_planilha_quantitativa(df_segmento):
    # Agrupar por faixa etária e classificação
    pivot = df_segmento.groupby(['Faixa Etaria', 'Classificacao'], observed=False).size().unstack(fill_value=0)
    
    # Garantir que as colunas existam mesmo se a contagem for 0
    for col in ['Titulares e dependentes', 'Agregados']:
        if col not in pivot.columns:
            pivot[col] = 0
            
    # Reordenar colunas
    pivot = pivot[['Titulares e dependentes', 'Agregados']]
    
    # Converter índice para string para poder adicionar 'Total' (já que é Categorical)
    pivot.index = pivot.index.astype(str)
    
    # Adicionar linha de total
    pivot.loc['Total'] = pivot.sum()
    
    return pivot

# Separar os dados por MÓDULO
df_apt = df_filtrado[df_filtrado['MÓDULO'] == 'AMB HOSP OBST APT']
df_enf = df_filtrado[df_filtrado['MÓDULO'] == 'AMB HOSP OBST ENF']

# Criar as planilhas
planilha_apt = criar_planilha_quantitativa(df_apt)
planilha_enf = criar_planilha_quantitativa(df_enf)

# Exibir os resultados
print("PLANILHA QUANTITATIVA - APT (AMB HOSP OBST APT)")
display(planilha_apt)
print("\n" + "="*50 + "\n")
print("PLANILHA QUANTITATIVA - ENF (AMB HOSP OBST ENF)")
display(planilha_enf)


In [ ]:
planilha_apt.to_excel('quanti_apt.xlsx', index=False)
planilha_enf.to_excel('quanti_enf.xlsx', index=False)